# Create music with model that we trained

## Necessary imports

In [ ]:
import tensorflow.keras as keras
import music21 as m21
import json

## Function

In [ ]:
def map_songs(songs_time_series, mapping):
    '''
    Converts the time series list of songs into a list of mapped songs
    
    Input params
    songs_time_series: list of songs in time series format
    mapping: dictionary with every note in songs_time_series mapped to an int
    
    Returns
    mapped_songs: list of songs mapped
    '''
    mapped_songs = []
    for song in songs_time_series:
        mapped_song = []
        for note in song:
            mapped_note = mapping[note]
            mapped_song.append(mapped_note)
        mapped_songs.append(mapped_song)
    return mapped_songs

## Load the model

In [ ]:
model = keras.models.load_model('model_3')

## Load the mapping

In [ ]:
with open('mapping_file2', 'r') as file:
    mapping_string = file.read().replace("\n    ", "")

In [ ]:
mapping = json.loads(mapping_string.replace("'", "\""))

## Input some random midi notes as start of song, map that and create test inputs

In [ ]:
song = [['72','74','64','55','62','60']]
mapped_song = map_songs(song, mapping)
test_inputs = keras.utils.to_categorical(mapped_song, num_classes=len(mapping))

## Create a song of 256 16th notes

In [ ]:
# Model predicts on the test_inputs that was created from our random song. Then it adds the prediction
# to the song and then predicts the next note on that new sequence.

for i in range(256-len(song)):
    note = model.predict(test_inputs).argmax()
    mapped_song[0].append(note)
    test_inputs = keras.utils.to_categorical(mapped_song, num_classes=len(mapping))

## Create a new song in midi notation with reversed mapping and the predicted song

In [ ]:
new_song = []
reverse_dict = {value: key for key, value in mapping.items()}
for i in mapped_song[0]:
    note = reverse_dict[i]
    new_song.append(note)

## Convert the song into a stream and save as midi

In [ ]:
stream = m21.stream.Stream()
start_symbol = None
step_counter = 1
for i, symbol in enumerate(new_song):
    if symbol != "-" or i + 1 == len(new_song):
        if start_symbol is not None:
            quarter_length_duration = 0.25 * step_counter # 0.25 * 4 = 1
            if start_symbol == "R":
                m21_event = m21.note.Rest(quarterLength=quarter_length_duration)
            else:
                m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)
            stream.append(m21_event)
            step_counter = 1
        start_symbol = symbol
    else:
        step_counter += 1
stream.write('midi', 'song3.mid')
